In [196]:
!pip install wget
!pip install calmap

In [197]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import wget
import math
import random
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode
import calmap
import folium
import warnings


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/population-by-country-2020/population_by_country_2020.csv


In [198]:
# Color pallete
Active, Recovered, Confirmed, Deceased =  '#ff073a', '#28a745', '#007bff', '#6c757d'

In [199]:
# remove existing files
! rm *.csv

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# download files
for url in urls:
    filename = wget.download(url)

In [200]:
confirmed_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [201]:
print(confirmed_df.shape)
print(deaths_df.shape)
print(recovered_df.shape)

(266, 109)
(266, 109)
(252, 109)


In [202]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1531,1703,1828,1939,2171,2335,2469,2704,2894,3224
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,726,736,750,766,773,782,789,795,803,820
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,3382,3517,3649,3848,4006,4154,4295,4474,4648,4838
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,738,743,743,743,745,745,747,748,750,751
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,26,27,27,27,27,30,35,35,35,36


In [203]:
deaths_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,50,57,58,60,64,68,72,85,90,95
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,28,28,30,30,31,31,31,31,31,31
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,425,432,437,444,450,453,459,463,465,470
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,40,40,41,42,42,43,44,45,45,46
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [204]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,207,220,228,252,260,310,331,345,397,421
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,410,422,431,455,470,488,519,531,543,570
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1508,1558,1651,1702,1779,1821,1872,1936,1998,2067
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,344,385,398,423,468,468,472,493,499,514
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,6,6,6,7,7,11,11,11,11,11


In [205]:
dates = confirmed_df.columns[4:]

confirmed_df_long = confirmed_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')
deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')
recovered_df_long = recovered_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

print(conf_df_long.shape)
print(deaths_df_long.shape)
print(recv_df_long.shape)

(27930, 6)
(27930, 6)
(26355, 6)


In [206]:
full_table = pd.merge(left=confirmed_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recovered_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,0.0
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,0.0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,0.0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,0.0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,0.0


In [207]:
print(full_table.shape)

(27930, 8)


In [208]:
full_table.isna().sum()

Province/State    19320
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered          2940
dtype: int64

In [209]:
full_table['Recovered'] = full_table['Recovered'].fillna(0)
full_table['Recovered'] = full_table['Recovered'].astype('int')
full_table.isna().sum()

Province/State    19320
Country/Region        0
Lat                   0
Long                  0
Date                  0
Confirmed             0
Deaths                0
Recovered             0
dtype: int64

In [210]:
full_table['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

In [211]:
full_table['Province/State'].unique()

array([nan, 'Australian Capital Territory', 'New South Wales',
       'Northern Territory', 'Queensland', 'South Australia', 'Tasmania',
       'Victoria', 'Western Australia', 'Alberta', 'British Columbia',
       'Grand Princess', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Nova Scotia', 'Ontario',
       'Prince Edward Island', 'Quebec', 'Saskatchewan', 'Anhui',
       'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 'Guangxi',
       'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hong Kong',
       'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin',
       'Liaoning', 'Macau', 'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong',
       'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Xinjiang',
       'Yunnan', 'Zhejiang', 'Faroe Islands', 'Greenland',
       'French Guiana', 'French Polynesia', 'Guadeloupe', 'Mayotte',
       'New Caledonia', 'Reunion', 'Saint Barthelemy', 'St Martin',
       'Martinique', 'Aruba', 'Curacao', 'Sint

In [212]:
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')
full_table['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

In [221]:
full_table = full_table[full_table['Province/State'].str.contains('Recovered')!=True]
full_table = full_table[full_table['Province/State'].str.contains('Grand Princess')!=True]
full_table = full_table[full_table['Province/State'].str.contains('Diamond Princess')!=True]
full_table['Province/State'].unique()

In [214]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']
full_table.sample(10)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
23376,NaN,Mozambique,-18.665695,35.529562,4/18/20,35,0,0,35
27394,NaN,Sao Tome and Principe,0.186360,6.613081,5/3/20,16,1,4,11
26802,NaN,Sri Lanka,7.000000,81.000000,5/1/20,690,7,162,521
13998,Aruba,Netherlands,12.518600,-70.035800,3/14/20,2,0,0,2
19973,NaN,Belgium,50.833300,4.000000,4/6/20,20814,1632,3986,15196
25899,NaN,Ecuador,-1.831200,-78.183400,4/28/20,24258,871,1557,21830
22355,Queensland,Australia,-28.016700,153.400000,4/15/20,999,5,442,552
26728,NaN,Honduras,15.200000,-86.241900,5/1/20,804,75,112,617
13339,New Brunswick,Canada,46.565300,-66.461900,3/12/20,1,0,0,1
25049,Saskatchewan,Canada,52.939900,-106.450900,4/25/20,349,4,0,345


In [236]:
full_table['Date'] = pd.to_datetime(full_table.Date)
full_table.sort_values(by=['Date'], inplace=True)
full_table.head(50)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0,0
168,Sint Maarten,Netherlands,18.0425,-63.0548,2020-01-22,0,0,0,0
169,NaN,Netherlands,52.1326,5.2913,2020-01-22,0,0,0,0
170,NaN,New Zealand,-40.9006,174.8860,2020-01-22,0,0,0,0
171,NaN,Nicaragua,12.8654,-85.2072,2020-01-22,0,0,0,0
172,NaN,Niger,17.6078,8.0817,2020-01-22,0,0,0,0
173,NaN,Nigeria,9.0820,8.6753,2020-01-22,0,0,0,0
174,NaN,North Macedonia,41.6086,21.7453,2020-01-22,0,0,0,0
175,NaN,Norway,60.4720,8.4689,2020-01-22,0,0,0,0
176,NaN,Oman,21.0000,57.0000,2020-01-22,0,0,0,0


In [215]:
full_table.to_csv('covid_19_cleaned.csv', index=False)

In [216]:
# Grouped by day, country
# =======================

full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

# new cases ======================================================
temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()

mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

# renaming columns
temp.columns = ['Country/Region', 'Date', 'No_Of_New_Cases', 'No_Of_New_Deaths', 'No_Of_New_Recovered']
# =================================================================

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['No_Of_New_Cases', 'No_Of_New_Deaths', 'No_Of_New_Recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['No_Of_New_Cases'] = full_grouped['No_Of_New_Cases'].apply(lambda x: 0 if x<0 else x)

full_grouped.sample(20)

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,No_Of_New_Cases,No_Of_New_Deaths,No_Of_New_Recovered
17203,4/29/20,Zimbabwe,32,4,5,23,0,0,0
10366,3/24/20,Iran,24811,1934,8913,13964,1762,122,982
6821,2/7/20,Kazakhstan,0,0,0,0,0,0,0
8254,3/14/20,Burkina Faso,2,0,0,2,0,0,0
7569,3/10/20,Kazakhstan,0,0,0,0,0,0,0
2308,2/11/20,Gambia,0,0,0,0,0,0,0
17676,4/4/20,Libya,18,1,0,17,0,-2,-18
4111,2/2/20,Yemen,0,0,0,0,0,0,0
10256,3/23/20,Sri Lanka,97,0,3,94,15,0,0
12868,3/8/20,Slovenia,16,0,0,16,9,0,0


In [217]:
full_grouped.to_csv('covid_19_country_wise.csv', index=False)

In [218]:
temp = full_table.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)

melted_temp = temp.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Recovered'])
fig = px.treemap(melted_temp, path=["variable"], values="value", height=250, width=1200,
                 color_discrete_sequence=[Active, Recovered, Deceased])
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [219]:
fig = px.pie(melted_temp, values="value", height=750, names='variable', title='Covid 19',
                 color_discrete_sequence=[Active, Recovered, Deceased])
fig.data[0].textinfo = 'label+text+value'
fig.show()

In [234]:
temp = full_grouped.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

fig = px.area(temp, x="Date", y="Count", color='Case', height=600,
             title='Cases over time', color_discrete_sequence = [Recovered, Deceased, Active])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

,Date,Case,Count
0,2020-01-22,Recovered,28
105,2020-01-22,Deaths,17
210,2020-01-22,Active,510
1,2020-01-23,Recovered,30
106,2020-01-23,Deaths,18
211,2020-01-23,Active,606
2,2020-01-24,Recovered,35
107,2020-01-24,Deaths,26
212,2020-01-24,Active,880
108,2020-01-25,Deaths,42
